<a href="https://colab.research.google.com/github/Prajna1999/eng-or-mt/blob/master/sft_full_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers sentencepiece  huggingface_hub datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer
base_mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", use_fast=False)

In [ ]:
import unicodedata
import regex
class OdiaTokenizerWrapper:
    def __init__(
        self,
        base_tokenizer,
        suffixes=None,
        named_entities=None,
        compound_roots=None,
        force_suffix_split=False,
        enable_fuzzy_split=False
        ):
      self.tokenizer=base_tokenizer
      self.vocab=self.tokenizer.get_vocab()

      # known Odia suffixes. In future expand the list
      self.suffixes=suffixes or[ "ମାନେ", "ଗୁଡ଼ିକ", "ରେ", "କୁ", "ରୁ", "ଟି", "ଙ୍କ","ଙ୍କୁ"]
      self.named_entities=named_entities or ["ଜଗନ୍ନାଥ", "ଭୁବନେଶ୍ୱର", "କଟକ", "ପୁରୀ", "ବାଲେଶ୍ୱର", "ସମ୍ବଲପୁର",
    "ଓଡ଼ିଶା", "ଝାରସୁଗୁଡ଼ା", "ଗଞ୍ଜାମ", "ମୟୂରଭଞ୍ଜ", "ନୟାଗଡ଼", "କଳାହାଣ୍ଡି",
    "ଖୋର୍ଦ୍ଧା", "ନୂଆପଡା", "ନବରଙ୍ଗପୁର", "ସୁନ୍ଦରଗଡ଼", "ବରଗଡ଼", "କନ୍ଧମାଳ",
    "ଗଞ୍ଜେଶ୍ୱର", "ଗୋପାଳପୁର", "ପାରାଦୀପ", "ଅନୁଗୁଳ", "ଦେବଗଡ଼", "କେନ୍ଦୁଝର",
    "ବୌଦ୍ଧ", "ମଲକାନଗିରି", "ଗଣେଶ", "ଶିବ", "ଲିଙ୍ଗରାଜ", "ମାଓ", "ମହାନଦୀ",
    "ଚିଲିକା", "ରଥଯାତ୍ରା", "କୋଣାର୍କ", "ସୁର୍ଯ୍ୟମନ୍ଦିର", "ଧୂଳିଗିରି", "ନନ୍ଦନକାନନ",
    "ଗଣେଶପୂଜା", "ଦୁର୍ଗାପୂଜା", "କାଳୀପୂଜା", "ଲକ୍ଷ୍ମୀପୂଜା", "ସରସ୍ୱତୀପୂଜା",
    "ପ୍ରଭାତୀ", "ମହାପ୍ରଭୁ", "ମାଁତରିଣୀ", "ମାଁସାମଲେଶ୍ୱରୀ", "ମାଁମଣିକେଶ୍ୱରୀ",
    "ମାଁକାନକଦୁର୍ଗା", "ବିମଳା", "ଭାଗବତ", "ଗୀତା", "ରାମାୟଣ", "ମହାଭାରତ",
    "ଗୋପବନ୍ଧୁ", "ବିଜୁପଟ୍ନାୟକ", "ନବୀନପଟ୍ନାୟକ", "ହରିହର", "ବିରାଜା", "ଚନ୍ଦ୍ରଭାଗା",
    "ମହେନ୍ଦ୍ରତନୟ", "ଅଶୋକ", "ଧରିତ୍ରୀ", "ସାମ୍ବ", "ନୀଳାଚଳ", "ପଟିଆ", "ନୟାପଲ୍ଲୀ"]
      self.compound_roots=compound_roots or [

                                            "ଶିକ୍ଷା", "ନୀତି", "ସଂସ୍କୃତି", "କେନ୍ଦ୍ର", "ବିଭାଗ", "ପୁସ୍ତକ", "ମେଳା",
    "ପ୍ରଶାସନ", "ନିୟମ", "ପରୀକ୍ଷା", "ମନ୍ତ୍ରଣାଳୟ", "ପ୍ରଧାନ", "ମନ୍ତ୍ରୀ",
    "ବିଶ୍ୱବିଦ୍ୟାଳୟ", "ସଂସ୍ଥା", "ନିୟୋଜନ", "ବିଭାଗ", "ସମିତି", "ଅଧିକାରୀ",
    "ଚିକିତ୍ସା", "ପ୍ରତିଷ୍ଠାନ", "ଶିଶୁ", "ସେବା", "ଉଦ୍ୟୋଗ", "ଆୟୋଜନ",
    "ତଥ୍ୟ", "ପ୍ରଯୁକ୍ତି", "ଖେଳ", "କ୍ରୀଡା", "ଅଭିଯାନ", "ଅନୁଷ୍ଠାନ",
    "ସାଂସ୍କୃତିକ", "ବିଜ୍ଞାନ", "ତନ୍ତ୍ର", "ଅନୁସନ୍ଧାନ", "ପ୍ରକାଶନ",
    "ବ୍ୟବସ୍ଥା", "ବଜେଟ", "ଉଦ୍‌ଘାଟନ", "ଅଧିବେଶନ", "ବୈଠକ", "ଉପସ୍ଥାପନ",
    "ନୀତି", "ଯୋଜନା", "ପଦକ୍ଷେପ", "ପ୍ରତିବେଦନ", "ସମୀକ୍ଷା", "ସୂଚନା",
    "ସୁରକ୍ଷା", "ନିର୍ବାଚନ", "ପ୍ରଚାର", "ନିୟମାବଳୀ", "ନିୟମକ", "ସଚିବ",
    "ପରିଚାଳନା", "ପାଠ୍ୟକ୍ରମ", "ଅନୁମୋଦନ", "ଶ୍ରମ", "ଉନ୍ନତି", "ଅନୁଶାସନ"
      ]
      self.force_suffix_split=force_suffix_split
      self.enable_fuzzy_split=enable_fuzzy_split

    def normalize_unicode(self, text):
      normalized_text=unicodedata.normalize("NFC", text)
      cleaned = normalized_text.replace('\u200d', '').replace('\u200c', '')
      return cleaned


    def is_known_token(self,token):
      # sentencepiece tokenizer sometimes have "__token" structure in their vocab
      return token in self.vocab or f"__{token}" in self.vocab

    def get_graphemes(self,word):
      return regex.findall(r'\X', word)

    def is_named_entity(self, word):
      for entity in self.named_entities:
          if self.enable_fuzzy_split:
              if word.startswith(entity) or word == entity:
                  return True
          else:
              if word == entity:
                  return True
      return False

    def compound_splitter(self,word):
      graphemes=self.get_graphemes(word)
      for i in range(2,len(graphemes)):
        left="".join(graphemes[:i])
        right="".join(graphemes[i:])

        if left in self.compound_roots and right in self.compound_roots:
          # print(f"Compound root split: '{word}' → '{left}' + '{right}'")
          return f"{left} {right}"
      return word

    def split_hyphenated(self, word):
      if "-" in word:
          parts = word.split("-")
          if len(parts) == 2 and all(len(p) > 1 for p in parts):
              # print(f"🔨 Hyphen split: '{word}' → '{parts[0]} {parts[1]}'")
              return f"{parts[0]} {parts[1]}"
      return word

    def split_suffix(self, word, depth=0):
        indent = "  " * depth
        graphemes = self.get_graphemes(word)

        for i in range(1, len(graphemes)):
            stem = "".join(graphemes[:i])
            suffix = "".join(graphemes[i:])

            if suffix in self.suffixes and len(stem) > 1:
              # disable unknown token check on an ad hoc basis
                if self.force_suffix_split or  not self.is_known_token(stem) or not self.is_known_token(suffix):

                    # print(f"{indent}📍 Split (grapheme-safe): '{word}' → '{stem}' + '{suffix}'")
                    split_stem = self.split_suffix(stem, depth + 1)
                    return f"{split_stem} {suffix}"
                else:
                    print(f"{indent}⚠️ Not splitting: '{word}' → parts unknown")
        print(f"{indent}✅ Final stem: '{word}'")
        return word


    def preprocess_text(self,text):
      normalized=self.normalize_unicode(text)
      words=normalized.split()

      processed=[]
      for word in words:

        if self.is_named_entity(word):
            print(f"Skipping named entity(fuzzy): {word}")
            processed.append(word)
        else:
            word = self.split_hyphenated(word)
            word = self.compound_splitter(word)
            word = self.split_suffix(word)
            processed.append(word)
      return " ".join(processed)

    def visualize_token_split(self, text, return_stats=False):
      print("\n🧪 ORIGINAL INPUT")
      print("📌 Text:", text)

      # --- Raw tokenization ---
      raw_tokens = self.tokenizer.tokenize(text)
      print("\n🧩 RAW TOKENIZATION")
      print(f"🧱 Tokens ({len(raw_tokens)}):", raw_tokens)

      # --- Preprocessing ---
      split_text = self.preprocess_text(text)

      # --- Preprocessed tokenization ---
      pre_tokens = self.tokenizer.tokenize(split_text)
      print("\n🛠️ AFTER PREPROCESSING")
      print("🧼 Cleaned:", split_text)
      print(f"🔤 Tokens ({len(pre_tokens)}):", pre_tokens)

      # --- Comparison ---
      print("\n📊 TOKEN COUNT COMPARISON")
      delta = len(raw_tokens) - len(pre_tokens)
      if delta > 0:
          print(f"✅ Token count reduced by: {delta} tokens")
      elif delta < 0:
          print(f"⚠️ Token count increased by: {-delta} tokens")
      else:
          print("➖ Token count unchanged")

      print("—" * 50)

      if return_stats:
          return {
              "original": text,
              "preprocessed": split_text,
              "raw_tokens": raw_tokens,
              "pre_tokens": pre_tokens,
              "delta": delta
          }

    def encode(self, text, **kwargs):
      preprocessed=self.preprocess_text(text)
      encoded=self.tokenizer.encode(preprocessed, **kwargs)
      return encoded


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# clean the dataframe fields for odia and english
import re
def clean_english(text):
  if not isinstance(text,str) or pd.isna(text):
    return ""

  text=unicodedata.normalize('NFKC', text)

  text=re.sub(r'\s+',' ',  text)
      # Remove URLs
  text = re.sub(r'https?://\S+|www\.\S+', '', text)

  # Remove HTML tags
  text = re.sub(r'<.*?>', '', text)

  # Standardize punctuation spacing
  text = re.sub(r'([.,!?()])', r' \1 ', text)
  text = re.sub(r'\s+', ' ', text).strip()

  return text

In [ ]:
odia_tokenizer=OdiaTokenizerWrapper(base_tokenizer=base_mistral_tokenizer)
def preprocess_dataframe(df):
  df_clean=df.copy()

  df_clean['eng_string_clean']=df_clean['eng_string'].apply(clean_english)
  df_clean['or_string_clean']=df_clean['or_string'].apply(odia_tokenizer.preprocess_text)

  df_clean=df_clean.reset_index(drop=True)
  return df_clean

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/bpcc_eng_odia.csv')
cleaned_df=preprocess_dataframe(df)

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

def convert_to_huggingface_dataset(df, test_size=0.1, val_size=0.1, random_state=42):
  df_copy=df.copy()

  train_df,test_df=train_test_split(df_copy, test_size=test_size, random_state=random_state)
  train_df,val_df=train_test_split(train_df, test_size=test_size, random_state=random_state)

  dataset_dict=DatasetDict({
      "train":Dataset.from_pandas(train_df),
      "validation":Dataset.from_pandas(val_df),
      "test":Dataset.from_pandas(test_df)
  })

  return dataset_dict


In [ ]:
hf_dataset=convert_to_huggingface_dataset(cleaned_df)

In [ ]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['eng_string', 'or_string', 'eng_string_clean', 'or_string_clean', '__index_level_0__'],
        num_rows: 73256
    })
    validation: Dataset({
        features: ['eng_string', 'or_string', 'eng_string_clean', 'or_string_clean', '__index_level_0__'],
        num_rows: 8140
    })
    test: Dataset({
        features: ['eng_string', 'or_string', 'eng_string_clean', 'or_string_clean', '__index_level_0__'],
        num_rows: 9044
    })
})

In [ ]:
# format for finetuning translation
import json
def format_for_translation(example, template_format="chatml"):
  eng_text=example['eng_string_clean']
  or_text=example['or_string_clean']
  if template_format=="chatml":
    formatted={
         "messages": [
                {"role": "system", "content": "You are a helpful translation assistant that translates English to Odia."},
                {"role": "user", "content": f"Translate the following English text to Odia: {eng_text}"},
                {"role": "assistant", "content": or_text}
            ]
    }
  elif template_format=="alpaca":
        # Alpaca format
        formatted = {
            "instruction": "Translate the following English text to Odia.",
            "input": eng_text,
            "output": or_text
        }
  elif template_format=="llama2":
    formatted={
        "text": f"<s>[INST] <<SYS>>\nYou are a helpful translation assistant that translates English to Odia.\n<</SYS>>\n\nTranslate the following English text to Odia: {eng_text} [/INST] {or_text}</s>"
    }
  elif template_format=="text_only":
    formatted={
         "text": f"English: {eng_text}\nOdia: {or_text}"
    }

  return formatted


def convert_to_jsonl(dataset, output_path, template_format="chatml"):
  with open(output_path, 'w', encoding='utf-8') as f:
    for example in dataset:
      # print(row['eng_string_clean'], row['or_string_clean'])
      formatted=format_for_translation(example, template_format)
      f.write(json.dumps(formatted, ensure_ascii=False)+'\n')
  print(f"Saved {len(dataset)} examples to {output_path}")

In [ ]:
import os
def prepare_all_formats(dataset_dict, output_dir="/content/drive/My Drive/formatted_data"):
  os.makedirs(output_dir, exist_ok=True)

  format_options=["chatml", "alpaca", "llama2", "text_only"]

  for split_name, dataset in dataset_dict.items():
    for fmt in format_options:
      output_path=f"{output_dir}/{split_name}_{fmt}.jsonl"
      convert_to_jsonl(dataset, output_path, template_format=fmt)
  print(f"All dataset formats saved to {output_dir}")


In [ ]:
prepare_all_formats(hf_dataset)

Saved 73256 examples to /content/drive/My Drive/formatted_data/train_chatml.jsonl
Saved 73256 examples to /content/drive/My Drive/formatted_data/train_alpaca.jsonl
Saved 73256 examples to /content/drive/My Drive/formatted_data/train_llama2.jsonl
Saved 73256 examples to /content/drive/My Drive/formatted_data/train_text_only.jsonl
Saved 8140 examples to /content/drive/My Drive/formatted_data/validation_chatml.jsonl
Saved 8140 examples to /content/drive/My Drive/formatted_data/validation_alpaca.jsonl
Saved 8140 examples to /content/drive/My Drive/formatted_data/validation_llama2.jsonl
Saved 8140 examples to /content/drive/My Drive/formatted_data/validation_text_only.jsonl
Saved 9044 examples to /content/drive/My Drive/formatted_data/test_chatml.jsonl
Saved 9044 examples to /content/drive/My Drive/formatted_data/test_alpaca.jsonl
Saved 9044 examples to /content/drive/My Drive/formatted_data/test_llama2.jsonl
Saved 9044 examples to /content/drive/My Drive/formatted_data/test_text_only.jsonl

In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install peft

In [ ]:
# Preprocessing the instructions
def preprocess_function(examples,tokenizer, max_length=512):
  # tokenize inputs and targets
  if "text" in examples:
    texts=examples["text"]
    model_inputs=tokenizer(texts, max_length=max_length, truncation=True)
    return model_inputs

  else:
    model_inputs={}
    for i in range(len(examples["instruction"])):
      text = f"### Instruction: {examples['instruction'][i]}\n"
      text += f"### Input: {examples['input'][i]}\n"
      text += f"### Response: {examples['output'][i]}"
      # tokenize the dataset
      tokenized=tokenizer(text, max_length=max_length, truncation=True)

      # Add to batch
      for k, v in tokenized.items():
        if k not in model_inputs:
          model_inputs[k]=[]
        model_inputs[k].append(v)
    return model_inputs


In [ ]:
import os
import logging
import torch
import numpy as np

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)

from datasets import load_dataset
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.INFO
)

# core SFT logic block
def train_translation_model(
    model_name,
    dataset_path,
    output_dir,
    format_type="llama2",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-04,
    bf16=True,
    logging_steps=10,
    save_steps=200,
    eval_steps=200,
    lora_r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    use_peft=True
):
  logging.info("Creating output directory at %s", output_dir)
  os.makedirs(output_dir,exist_ok=True)

  # load tokenizer
  logging.info("Loading tokeniser from %s",model_name)
  tokenizer=AutoTokenizer.from_pretrained(model_name, padding_side="right",use_fast=False)
  if tokenizer.pad_token is None:
    tokenizer.pad_token=tokenizer.eos_token


  # load the model for full fine tuning
  logging.info("Loading model from %s", model_name)
  if bf16 and torch.cuda.is_available():
    bnb_config=BitsAndBytesConfig(load_in_8bit=True)
    model_output=AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=bnb_config,
        output_loading_info=True
    )
    if isinstance(model_output, tuple):
      model=model_output[0]
    else:
      model=model_output
  else:
    model_output=AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True, output_loading_info=True)
    if isinstance(model_output, tuple):
      model = model_output[0]
    else:
      model = model_output

  if use_peft:
    from peft import LoraConfig, get_peft_model, TaskType
    logging.info("Applying LoRA config for PEFT")
    lora_config=LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj"
                    , "down_proj", "lm_head"]

    )
    model=get_peft_model(model, lora_config)
    model.print_trainable_parameters()
  # load the dataset from Google drive
  logging.info("Loading data from %s", dataset_path)
  dataset=load_dataset("json",data_files=dataset_path)

  # if no explicit validation sets are provided in the loaded data
  if "validation" not in dataset:
    logging.info("Splitting dataset into training and validation sets")
    dataset=dataset["train"].train_test_split(test_size=0.1)

  # preprocess the datarset
  logging.info("Preprocessing the dataset")
  tokenized_dataset=dataset.map(
      lambda examples: preprocess_function(examples, tokenizer),
      batched=True,
      remove_columns=dataset["train"].column_names
  )

  # prepare the data collator
  logging.info("Prepare the data collator")
  data_collator=DataCollatorForLanguageModeling(
      tokenizer=tokenizer,
      mlm=False

  )
  # training args. Note the seq2seq models are probably not supported for Mistral class of models.
  # hence going for normal trainers
  logging.info("Setting up training arguments")
  training_args=TrainingArguments(
      output_dir=output_dir,
      num_train_epochs=num_train_epochs,
      per_device_train_batch_size=per_device_train_batch_size,
      per_device_eval_batch_size=per_device_train_batch_size,
      gradient_accumulation_steps=gradient_accumulation_steps,
      learning_rate=learning_rate,
      weight_decay=0.01,
      bf16=bf16,
      logging_steps=logging_steps,
      save_steps=save_steps,
      eval_steps=eval_steps,
      save_total_limit=3,
      evaluation_strategy="steps",
      load_best_model_at_end=True,
      report_to="tensorboard",
      push_to_hub=False,
      gradient_checkpointing=True
  )

  # init trainer
  trainer=Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_dataset["train"],
      eval_dataset=tokenized_dataset["validation"] if "validation" in tokenized_dataset else tokenized_dataset["test"],
      tokenizer=tokenizer,
      data_collator=data_collator
  )
   # Start training
  logging.info("Starting training")
  trainer.train()
  logging.info("Training completed")

  # Save the fine-tuned model and tokenizer
  logging.info("Saving the model and tokenizer to %s", output_dir)
  trainer.save_model(output_dir)
  tokenizer.save_pretrained(output_dir)
  logging.info("Model and tokenizer saved successfully")

  return model, tokenizer



In [ ]:
def main():
    # Hard-coded parameters for Google Colab environment
    model_name = "mistralai/Mistral-7B-Instruct-v0.3"  # Change this if you prefer another model
    # Set this to the path of your JSONL formatted dataset in Google Drive
    dataset_path = "/content/drive/My Drive/formatted_data/train_llama2.jsonl"
    # Set the output directory in your Google Drive
    output_dir = "/content/drive/My Drive/mistral-7b-ft-eng-or-translation"
    format_type = "llama2"
    num_train_epochs = 3
    per_device_train_batch_size = 4
    gradient_accumulation_steps = 16
    learning_rate = 2e-4
    bf16 = True
    logging_steps = 10
    save_steps = 200
    eval_steps = 200

    logging.info("Starting full fine-tuning with the following parameters:")
    logging.info("Model: %s", model_name)
    logging.info("Dataset Path: %s", dataset_path)
    logging.info("Output Directory: %s", output_dir)

    model,tokenizer=train_translation_model(
        model_name=model_name,
        dataset_path=dataset_path,
        output_dir=output_dir,
        format_type=format_type,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        bf16=bf16,
        logging_steps=logging_steps,
        save_steps=save_steps,
        eval_steps=eval_steps,

    )

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
from peft import PeftModel
def merge_lora_model(model_path, output_path, base_model_name):
    """
    Loads the LoRA–finetuned model, merges the LoRA weights into the base model,
    and saves the merged model.

    Args:
        model_path (str): Path to the directory of the LoRA–finetuned model.
        output_path (str): Directory to save the merged model.
        base_model_name (str): Name or path of the base model (used for initialization).
    """
    # Load the base model
    print(f"Loading base model {base_model_name} ...")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        trust_remote_code=True,
        device_map="auto"
    )

    # Load the fine-tuned LoRA model.
    print(f"Loading LoRA model from {model_path} ...")
    model = PeftModel.from_pretrained(base_model, model_path)

    # Merge the LoRA weights into the base model.
    print("Merging LoRA weights into the base model ...")
    model = model.merge_and_unload()

    # Save the merged model to the specified output directory.
    print(f"Saving the merged model to {output_path} ...")
    os.makedirs(output_path, exist_ok=True)
    model.save_pretrained(output_path)
    model.push_to_hub("Prajna1999/mistral-7b-translation-finetuned")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.save_pretrained(output_path)
    tokenizer.push_to_hub("Prajna1999/mistral-7b-translation-finetuned")
    print(f"Merged model saved at {output_path}")

In [ ]:
if __name__ == "__main__":
    # Hard-coded parameters for Google Colab
    base_model_name = "mistralai/Mistral-7B-Instruct-v0.3"  # Change as needed
    model_path = "/content/drive/My Drive/mistral-7b-ft-eng-or-translation"  # LoRA–finetuned model path
    output_path = "/content/drive/My Drive/mistral-7b-ft-eng-or-translation-merged-model"  # Path to save the merged model

    merge_lora_model(model_path, output_path, base_model_name)

In [ ]:
from transformers import pipeline

